In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_756474/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_31_7,0.999850,0.705710,0.658596,0.999808,0.000063,0.174703,0.239160,0.000070,0.003796,0.007910,0.999699,0.007971,149.358721,335.748460,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_31_6,0.999850,0.705708,0.658580,0.999810,0.000063,0.174705,0.239172,0.000069,0.003796,0.007912,0.999698,0.007973,149.357594,335.747333,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_31_5,0.999850,0.705692,0.658498,0.999811,0.000063,0.174714,0.239229,0.000069,0.003796,0.007913,0.999698,0.007975,149.357032,335.746771,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_31_4,0.999850,0.705699,0.658591,0.999814,0.000063,0.174710,0.239164,0.000068,0.003797,0.007915,0.999698,0.007976,149.356200,335.745939,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_31_3,0.999850,0.705704,0.658493,0.999815,0.000063,0.174707,0.239233,0.000067,0.003799,0.007919,0.999698,0.007980,149.354179,335.743918,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.542158,0.384278,0.334329,0.447644,0.191508,0.365519,0.466317,0.207598,0.228212,0.437616,0.077162,0.441022,133.305650,319.695390,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.477664,0.358956,0.300883,0.321052,0.218485,0.380551,0.489746,0.255176,0.224312,0.467424,-0.052833,0.471061,133.042078,319.431817,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.414606,0.273932,0.306064,0.385414,0.244861,0.431026,0.486116,0.230986,0.290621,0.494834,-0.179935,0.498685,132.814129,319.203868,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.253916,0.134828,0.156728,0.216698,0.312075,0.513604,0.590730,0.294397,0.333322,0.558637,-0.503825,0.562984,132.329024,318.718764,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
